In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('data/DailyDelhiClimateTrain.csv')
df_test = pd.read_csv('data/DailyDelhiClimateTest.csv')

In [3]:
df_train.shape

(1462, 5)

In [4]:
df_test.shape

(114, 5)